In [4]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [5]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [6]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_3636.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 1. TURN: 1 


You raise your hand confidently and bid $100. The crowd murmurs, sizing up the competition and the storage's visible contents. Another bidder, a woman with keen eyes, immediately raises the bid to $150.

What is your next move?

possible actions:
1. Bid $200.
2. Bid $250.
3. Drop out of the bidding.

I select option 2. TURN: 2 


You counter swiftly, raising the bid to $250. The auction atmosphere thickens with anticipation. The woman hesitates for a moment but then, with a determined nod, she raises the bid to $300. 

A few bidders drop out, leaving only a handful, including you, in the running. The storage's hidden treasures seem to promise more than meets the eye, judging by the eagerness in the remaining bidders' eyes.

What is your next move?

possible actions:
1. Bid $350.
2. Bid $400.
3. Drop out of the bidding.

I select option 2. TURN: 3 


You take a deep breath and bid $400, pushing the limits of your budget. The woman pauses, her expression unreadable for a moment, then she shakes her head and steps back, effectively dropping out of the bidding. The other remaining bidders, after a moment's hesitation, also decide not to challenge your bid.

The auctioneer calls out, "Going once, going twice... and sold to the gentleman for $400!" The crowd disperses, some with looks of disappointment, others with shrugs of indifference.

Congratulations, you've won the bid for the storage room. As you approach to take a closer look at your newly acquired possessions, you can't help but wonder what secrets or treasures might be hidden within the dusty confines of the storage.

summary: Kyle Jackson wins the storage room auction with a bid of $400, beating out the competition. The round concludes with Kyle about to explore the contents of his newly acquired storage room, eager to discover what valuables or 


Memories:
"""
- Kyle Jackson wins the storage room auction with a bid of $400, beating out the competition. The round concludes with Kyle about to explore the contents of his newly acquired storage room, eager to discover what valuables or secrets it might hold.
"""


You follow the manager through the myriad hallways to his dimly lit office, where the furnishings and appliances remain unchanged since the 90’s. He often reminds you that the facility is in fact even older, and has been here since the 60’s. You made a quick phone call to Alex, a man you owe money to, and explained to him that you will pay him back in a few days once you have it, to which he responded with distasteful threats. The manager took out an old tin box and placed it on his desk and reached towards you with an open palm, gesturing for you to pay up, which you obliged. Upon handing you a key for your newly owned storage room, he urged you to watch some old security footage.

Through his CRT screen, he explains to

...

I select option 2. TURN: 1  (I'm insane!)


As you step cautiously behind Professor Dagger, the foul stench growing stronger with each step, you can't help but ask, "What is that dreadful smell?"

Professor Dagger pauses for a moment, turning slightly to glance back at you. His face is grim, his eyes reflecting a deep concern. "Decay," he answers succinctly. "Years, perhaps decades of it. This place... it's not just a storage unit. It's a tomb of sorts, for secrets and for things best left forgotten."

The revelation sends a chill down your spine, but your curiosity and the dire need to change your fortunes push you forward. The corridor stretches ahead, the air thick with the scent of secrets and decay.

As you proceed, the dim light from the entrance fades, leaving you in near darkness, save for the faint glow of Professor Dagger's flashlight. Shadows dance along the walls, and the atmosphere feels heavy with anticipation and dread.

What will you do next?

possible actions:
1. Conti

I select option 1. TURN: 2  (I'm insane!)


You decide to continue following Professor Dagger quietly, letting the weight of the situation and your own thoughts simmer in silence. The narrow corridor seems to stretch indefinitely, its end swallowed by darkness. The only sound is the soft echo of your footsteps and the occasional, unsettling drip of water from an unseen source. The air grows colder as you move deeper into the corridor, the sense of foreboding thickening with each step.

Finally, the corridor opens into a vast, hidden room. The air is stale, and the smell of decay is overpowering here. In the center of the room lies the withered body of a woman, her features obscured by time and the shadows that dance across the walls from the flickering light of a single, decrepit chandelier. Surrounding her body is a pentagram made of a strange red fluid, its color unnervingly vibrant against the dull grey of the concrete floor.

Professor Dagger steps cautiously into the room, his fla

I am making a SANITY check against my own sanity points: 5.
And... FAILED! I lost 5 sanity points.
I am shocked and lose control of my actions.
TURN: 3  (I'm insane!)


As the full horror of the scene before you sinks in, your mind reels from the shock. The sight of the withered body, the sinister pentagram, and the oppressive atmosphere of the room conspire to shatter the fragile barriers of your sanity. The world spins, and you feel an overwhelming sense of dread and despair clutching at your throat. Your sanity, already frayed by the experiences leading up to this moment, crumbles entirely under the weight of the eldritch horror before you.

In a state of panic and confusion, you stagger backwards, your actions no longer your own. Your vision blurs, and you find yourself screaming, though you can't hear your own voice over the ringing in your ears. The room seems to twist and warp around you, the shadows coming alive with unspeakable forms.

Professor Dagger turns towards you, his f

I select option 1. TURN: 4  (I'm insane!)


With every ounce of willpower you have left, you attempt to calm yourself, focusing on slowing your breathing and steadying your racing heart. The terror that gripped you so fiercely begins to ebb, replaced by a grim determination. You remind yourself why you're here, the desperate need for a new beginning, and the hope that this book could be the key to changing your fortunes.

Gradually, the room comes back into focus, and the oppressive atmosphere seems slightly less suffocating. Professor Dagger, noticing the change in your demeanor, gives you a nod of approval, though concern still lingers in his eyes.

"You're stronger than you know, Kyle," he says softly, his voice a grounding presence in the chaos of your thoughts. "Let's see this through, together."

Feeling somewhat steadied by his words, you nod, and together you approach the podium at the far end of the room. The book that rests upon it seems to call to you, its presence both allu

I select option 2. TURN: 5  (I'm insane!)


In a moment of sheer panic and survival instinct, you turn to flee the room, the image of Professor Dagger's final moments etched into your mind. The creature, once the withered body of a woman, now a thing of nightmares, seems to sense your intention to escape and quickens its pursuit with an unnatural speed.

Your heart pounds in your chest as you sprint back through the dark corridor, the echoes of your footsteps and the slithering sound of the creature close behind filling the air. The stench of decay is now mixed with the sharp tang of fear, every breath you take laced with desperation.

As you reach the entrance to the hidden room, you realize the gravity of your situation. The only exit from the storage facility is locked, a fact that dawns on you with crippling dread. The memory of the middle-aged woman who promised to unlock the door for you flashes through your mind, a glimmer of hope in the darkness.

You rush to the door, your han